# Decoding a neural network from a JSON file to Python, then encoding the hessian from laplace.py to a CSV (regression)

- Step 1: Read the dataset
- Step 2: Initialize model
- Step 3: Read the weights and biases from the JSON file, then load them into the previously initialized model
- Step 4: Generate the hessians and write them to CSV files

In [1]:
# imports
from laplace import Laplace

import numpy as np
import pandas as pd
import torch

import json

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(43)

### Step 1: Read the dataset

In [2]:
# Import data from csv

# Load data from CSV file using pandas
df = pd.read_csv('data_regression.csv')

# Split the dataframe into x and y tensors
data = torch.from_numpy(df[['xs', 'y']].to_numpy()).to(torch.float32)
x = torch.from_numpy(df['xs'].to_numpy()).to(torch.float32)
y = torch.from_numpy(df['y'].to_numpy()).to(torch.float32)

### Step 2: Initialize model

In [3]:
# Init model

n_hidden = 10
D = 1
out_dim = 1 
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Tanh(),
    nn.Linear(n_hidden, out_dim)
)

loss_fn = nn.CrossEntropyLoss()

### Step 3: Read the weights and biases from the JSON file, then load them into the previously initialized model

In [4]:
with open('nn_regression.json') as fin:
    nn_json_str = fin.read()
    nn_json = json.loads(nn_json_str)

nn_json

[{'bias': [-0.42261407,
   -0.21227753,
   -0.0096151605,
   -0.7529069,
   1.3016218,
   3.116596,
   -2.1422381,
   -1.2806743,
   0.78418183,
   -1.3360132],
  'weight': [[-0.55855876,
    0.2368329,
    0.7279194,
    1.1712134,
    -0.2849682,
    -0.82284534,
    0.76305974,
    0.28283465,
    -0.24539436,
    0.6470228]]},
 {'bias': [0.13576016],
  'weight': [[-0.27233723],
   [0.7866149],
   [0.26231468],
   [0.16347119],
   [-1.6750962],
   [1.7097319],
   [-1.3647312],
   [1.6296383],
   [-1.1626163],
   [-0.75578994]]}]

In [5]:
assert len(model.state_dict()) == 2 * len(nn_json)
iter_states = iter(model.state_dict())

# for layer in model.state_dict():
#     print(layer)
for layer_read in nn_json:
    state_w = next(iter_states)
    state_b = next(iter_states)
    tensor_w = torch.tensor(layer_read['weight']).T
    tensor_b = torch.tensor(layer_read['bias']).T
    model.state_dict()[state_w].data.copy_(tensor_w)
    model.state_dict()[state_b].data.copy_(tensor_b)


C:\Users\adeli\AppData\Local\Temp\ipykernel_28676\3249112778.py:10: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3281.)
  tensor_b = torch.tensor(layer_read['bias']).T


In [6]:
for val in model.state_dict().values():
    display(val)

tensor([[-0.5586],
        [ 0.2368],
        [ 0.7279],
        [ 1.1712],
        [-0.2850],
        [-0.8228],
        [ 0.7631],
        [ 0.2828],
        [-0.2454],
        [ 0.6470]])

tensor([-0.4226, -0.2123, -0.0096, -0.7529,  1.3016,  3.1166, -2.1422, -1.2807,
         0.7842, -1.3360])

tensor([[-0.2723,  0.7866,  0.2623,  0.1635, -1.6751,  1.7097, -1.3647,  1.6296,
         -1.1626, -0.7558]])

tensor([0.1358])

### Step 4: Generate the hessians and write them to CSV files

In [7]:
# we use the unsqueeze() method with the argument 1 to add an extra dimension
# this extra dimension converts each value into an array of its own
X = x.unsqueeze(1)

In [8]:
la = Laplace(model, 'regression',
             subset_of_weights='all',
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(X, y), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_regression_all_full_ggn.csv', array, delimiter=',')

c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl\myenv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [9]:
la = Laplace(model, 'regression',
             subset_of_weights='last_layer',
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(X, y), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_regression_ll_full_ggn.csv', array, delimiter=',')

In [10]:
la = Laplace(model, 'regression',
             subset_of_weights='subnetwork',
             subnetwork_indices = torch.LongTensor([1, 3, 5, 6, 7, 9]),
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(X, y), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_regression_subnet_full_ggn.csv', array, delimiter=',')